In [1]:
%%html
<style>
    #notebook-container{
    width: 100%
    }
</style>

In [2]:
import numpy as np

In [3]:
A = np.array([[1, 2, 3],
             [1, 2, 3],
              [1, -1, 0],
              [-1, -1, 4],
              [3, 1, -1],
              [2, 0, -2]])

In [9]:
Left = np.dot(np.linalg.inv(np.dot(A.T, A)), A.T)
Right = np.dot(A.T, np.linalg.inv(np.dot(A, A.T)))

LinAlgError: Singular matrix

In [6]:
np.dot(Left, A)

array([[ 1.00000000e+00,  2.77555756e-17, -1.38777878e-16],
       [-8.32667268e-17,  1.00000000e+00,  2.77555756e-16],
       [ 0.00000000e+00,  2.08166817e-17,  1.00000000e+00]])

In [11]:
np.linalg.norm(np.dot(A, Left) - np.identity(6))

1.7320508075688772

# Problem 3
## Task (c)

In [15]:
def richardson_step(x_old, A, b):
    return x_old - np.dot(A.T,(np.dot(A,x_old)-b))/(np.linalg.norm(A)**2)

def richardson_iter(x_old, A, b, acc):
    while True:
        x_new = richardson_step(x_old, A, b)
        if np.linalg.norm(np.dot(A.T,(np.dot(A,x_new)-b))) < acc:
            return x_new
            break
        
        x_old = x_new
    

In [21]:
np.random.seed(7)
A = np.random.rand(20,10)
b = np.random.rand(20,1)

x_hat = np.dot(np.linalg.pinv(A), b)

acc = 0.001
x0 = np.zeros((10,1))

solution = richardson_iter(x0, A, b, acc)

print(x_hat.round(5))
print(solution.round(5))
print(np.linalg.norm(x_hat - solution))

[[ 0.23137]
 [-0.38996]
 [ 0.85896]
 [ 0.45908]
 [ 0.17428]
 [-0.08715]
 [ 0.59027]
 [-0.13201]
 [ 0.12858]
 [-0.54262]]
[[ 0.22927]
 [-0.38974]
 [ 0.85575]
 [ 0.45602]
 [ 0.17581]
 [-0.0851 ]
 [ 0.58861]
 [-0.13036]
 [ 0.12894]
 [-0.5394 ]]
0.00682452837533685
